Ad:Cudi  
Soyad:Şami  
Okul Numarası:2012721308  
GitHub Repo Bağlantısı: https://github.com/kullanici_adi/MLP_Flask

##  Makine Öğrenmesi
### Proje 3: Medical Cost Prediction

Bu projede, bireylerin tıbbi özelliklerine göre sağlık harcamalarını
tahmin etmek amacıyla Çoklu Doğrusal Regresyon (Multiple Linear Regression)
modeli geliştirilmiştir.

Kullanılan veri seti; yaş, cinsiyet, vücut kitle indeksi (BMI),
sigara kullanımı, çocuk sayısı ve bölge gibi değişkenleri içermektedir.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os

os.listdir('/content/drive/MyDrive/Medical_Cost_Prediction')


['insurance.csv']

### Kullanılan Kütüphaneler

Bu projede veri analizi ve modelleme için aşağıdaki Python kütüphaneleri
kullanılmıştır.


In [6]:
import pandas as pd
import numpy as np


### Veri Setinin Yüklenmesi

Veri seti Google Drive üzerinden projeye dahil edilmiştir ve
pandas kütüphanesi kullanılarak okunmuştur.


In [7]:
df = pd.read_csv('/content/drive/MyDrive/Medical_Cost_Prediction/insurance.csv')
df.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Veri Setinin Genel İncelenmesi

Bu adımda veri setinin boyutu, sütun isimleri ve veri tipleri incelenmiştir.


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


### Eksik Veri Analizi

Veri setinde eksik değer olup olmadığı kontrol edilmiştir.


In [9]:
df.isnull().sum()


,0
age,0
sex,0
bmi,0
children,0
smoker,0
region,0
charges,0


### Kategorik Verilerin Kodlanması (Encoding)

Veri setinde sayısal olmayan (kategorik) değişkenler bulunmaktadır.
Bu değişkenlerin regresyon modelinde kullanılabilmesi için sayısal
değerlere dönüştürülmesi gerekmektedir.

Bu projede kategorik değişkenler için One-Hot Encoding yöntemi
kullanılmıştır. Bu yöntem, kategoriler arasında yapay bir sıralama
oluşturmadığı için tercih edilmiştir.


In [10]:
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()


,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,False,True,False,False,True
1,18,33.770,1,1725.55230,True,False,False,True,False
2,28,33.000,3,4449.46200,True,False,False,True,False
3,33,22.705,0,21984.47061,True,False,True,False,False
4,32,28.880,0,3866.85520,True,False,True,False,False


### Bağımlı ve Bağımsız Değişkenlerin Belirlenmesi

Bu çalışmada sağlık harcamaları (charges) bağımlı değişken (target)
olarak seçilmiştir.

Diğer tüm değişkenler bağımsız değişkenler (features) olarak
kullanılmıştır.


In [11]:
X = df_encoded.drop("charges", axis=1)
y = df_encoded["charges"]

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (1338, 8)
y shape: (1338,)


### Eğitim ve Test Verilerinin Ayrılması

Modelin performansını doğru şekilde değerlendirebilmek için
veri seti eğitim (%80) ve test (%20) olarak ikiye ayrılmıştır.


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


### Çoklu Doğrusal Regresyon Modelinin Oluşturulması

Bu aşamada, birden fazla bağımsız değişken kullanılarak
Multiple Linear Regression modeli oluşturulmuş ve eğitilmiştir.


In [15]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

### Model Performansının Değerlendirilmesi

Modelin başarımı R², Mean Absolute Error (MAE) ve
Mean Squared Error (MSE) metrikleri kullanılarak değerlendirilmiştir.


In [16]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("R² Score:", r2)
print("MAE:", mae)
print("MSE:", mse)


R² Score: 0.7835929767120724
MAE: 4181.194473753648
MSE: 33596915.851361446


### Geriye Doğru Eleme (Backward Elimination)

Bu aşamada, modelde kullanılan bağımsız değişkenlerin istatistiksel
olarak anlamlı olup olmadıkları p-değerleri yardımıyla incelenmiştir.

p-değeri 0.05'ten büyük olan değişkenler, model üzerinde anlamlı bir
etkiye sahip olmadıkları varsayılarak modelden çıkarılmıştır.
Bu işlem, tüm değişkenler istatistiksel olarak anlamlı olana kadar
tekrarlanmıştır.


In [17]:
import statsmodels.api as sm


In [24]:
import statsmodels.api as sm

X_be = df_encoded.drop("charges", axis=1)
y_be = df_encoded["charges"]

X_be = X_be.astype(float)

X_be = sm.add_constant(X_be)

model_sm = sm.OLS(y_be, X_be).fit()

print(model_sm.summary())


                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Tue, 23 Dec 2025   Prob (F-statistic):               0.00
Time:                        19:45:05   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.194e+04    987.819  

Backward Elimination uygulanmadan önce, One-Hot Encoding sonucunda oluşan
tüm değişkenler sayısal (float) veri tipine dönüştürülmüştür.
Bu işlem, istatistiksel modelleme sırasında oluşabilecek veri tipi
uyumsuzluklarını önlemek amacıyla yapılmıştır.


In [25]:
X_reduced = df_encoded.drop(
    columns=["charges", "sex_male", "region_northwest"]
)

y_reduced = df_encoded["charges"]


In [26]:
import statsmodels.api as sm

X_reduced = X_reduced.astype(float)
X_reduced = sm.add_constant(X_reduced)

model_sm_reduced = sm.OLS(y_reduced, X_reduced).fit()
print(model_sm_reduced.summary())


                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     668.3
Date:                Tue, 23 Dec 2025   Prob (F-statistic):               0.00
Time:                        19:49:41   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1331   BIC:                         2.715e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.217e+04    949.538  

Backward Elimination sonucunda, sex_male ve region_northwest değişkenlerinin
p-değerlerinin 0.05'ten büyük olduğu görülmüş ve bu değişkenler modelden
çıkarılmıştır. Kalan değişkenlerin tamamı istatistiksel olarak anlamlıdır.


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_reduced.drop("const", axis=1),
    y_reduced,
    test_size=0.2,
    random_state=42
)


In [28]:
from sklearn.linear_model import LinearRegression

final_model = LinearRegression()
final_model.fit(X_train, y_train)


LinearRegression()

In [29]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y_pred = final_model.predict(X_test)

print("Final R²:", r2_score(y_test, y_pred))
print("Final MAE:", mean_absolute_error(y_test, y_pred))
print("Final MSE:", mean_squared_error(y_test, y_pred))


Final R²: 0.7835205571421145
Final MAE: 4185.809142538226
Final MSE: 33608158.89773258


### Backward Elimination Öncesi ve Sonrası Karşılaştırma

Backward Elimination uygulanmadan önce ve sonra model performans metrikleri
karşılaştırılmıştır. R², MAE ve MSE değerlerinin neredeyse aynı kaldığı
gözlemlenmiştir. Bu sonuç, çıkarılan değişkenlerin model başarımına
anlamlı bir katkı sağlamadığını göstermektedir.

Dolayısıyla, daha az değişken içeren ve daha sade bir model elde edilmiştir.


### Modelin Genel Yorumu

Elde edilen sonuçlara göre sigara kullanımı, yaş, BMI ve çocuk sayısı
sağlık harcamalarını en fazla etkileyen faktörlerdir. Özellikle sigara
kullanımı değişkeninin katsayısının yüksek olması, bu faktörün sağlık
maliyetleri üzerindeki güçlü etkisini göstermektedir.


In [30]:
import pickle

model_path = "/content/drive/MyDrive/Medical_Cost_Prediction/model.pkl"

with open(model_path, "wb") as file:
    pickle.dump(final_model, file)

print("Model başarıyla kaydedildi.")


Model başarıyla kaydedildi.


### Modelin Kaydedilmesi

Eğitilen ve son hali verilen regresyon modeli, Flask uygulamasında
kullanılmak üzere `.pkl` formatında kaydedilmiştir.
